In [53]:
with open('input.txt') as f:
    lines = f.readlines()

seeds = lines.pop(0).strip()
seeds = seeds.split(':')[1].strip().split(' ')
seeds = [int(x) for x in seeds]

seeds_ranges = []
for i in range(0, len(seeds)-1, 2):
    seeds_ranges.append(range(seeds[i], seeds[i] + seeds[i+1]))


In [54]:
def str_to_range(str_):
    str_ = str_.split(' ')
    str_ = [int(x) for x in str_ if x]
    return str_

def return_next_value(value, range_str):
    range_n = str_to_range(range_str)
    if value in range(range_n[0], range_n[0] + range_n[2]):
        return value + (range_n[1] - range_n[0])
    else:
        return value
    
def return_next_map(loc, range_):
    next_loc = None
    for r in range_:
        next_loc_ = return_next_value(loc, r)
        if next_loc_ != loc:
            next_loc = next_loc_
    if not next_loc:
        next_loc = loc
    return next_loc

In [55]:
return_next_value(61, "60 56 37")

57

seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4

In [56]:
# Clean
maps = []
ranges = {}
for line in lines:
    line = line.strip().replace('\n', '').replace('map:', '').strip()
    if line:
        maps.append(line)

map_str = ''
for line in maps:
    if '-to-' in line:
        map_str = line
        ranges[map_str] = ''
        continue
    ranges[map_str] += '|' + line if ranges[map_str] else line

In [57]:
return_next_value(50, "50 98 2")

98

In [58]:
from multiprocessing import Pool
import os
from functools import partial

In [59]:
def get_lowest_loc(next_loc, ranges, seeds_ranges):
    loc = next_loc
    for k in reversed(list(ranges.keys())):
        range_str = ranges[k]
        range_str = range_str.split('|')
        next_loc = return_next_map(next_loc, range_str)
    for seed_range in seeds_ranges:
        if next_loc in seed_range:
            return loc

In [60]:
seeds_ranges

[range(1132132257, 1455563254),
 range(2043754183, 2048255238),
 range(2539071613, 3598100002),
 range(1695770806, 1756240975),
 range(2220296232, 2471712170),
 range(1673679740, 1679743438),
 range(962820135, 1096002452),
 range(262615889, 590396394),
 range(3602765034, 3797623755),
 range(2147281339, 2184747848)]

In [63]:
loc = range(0, 10000000000)
found = False

with Pool(os.cpu_count()) as p:
    partial_ = partial(get_lowest_loc, ranges=ranges, seeds_ranges=seeds_ranges)
    for result in p.imap_unordered(partial_, loc, chunksize=100000):
        if result:
            print(result)
            p.terminate()
            break

KeyboardInterrupt: 